<a href="https://colab.research.google.com/github/kodinkod/gpt3ru_sandbox/blob/main/fine_tune_gpt3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Начнем с импота необходимых библиотек 

In [1]:
import torch


In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 72.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 97.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 22.6 MB/s eta 0:00:00


In [4]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import TextDataset, DataCollatorForLanguageModeling

In [5]:
model_name = "sberbank-ai/rugpt3medium_based_on_gpt2" 
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

In [7]:

text = """

Ваш текст для предобучения модели.

"""

# Сохраним обучающие данные в .txt файл 
train_path = 'train_dataset.txt'
with open(train_path, "w") as f:
    f.write(text)



train_path = 'data_Ma.txt' # путь к датасету 

# Создание датасета
train_dataset = TextDataset(tokenizer=tokenizer,file_path=train_path,block_size=32)
  
# Создание даталодера (нарезает текст на оптимальные по длине куски)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [8]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./finetuned",         # The output directory
    overwrite_output_dir=True,        # Overwrite the content of the output dir
    num_train_epochs=40,              # number of training epochs
    per_device_train_batch_size=20,   # batch size for training
    per_device_eval_batch_size=32,    # batch size for evaluation
    warmup_steps=9,                   # number of warmup steps for learning rate scheduler
    gradient_accumulation_steps=5
    )

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    optimizers = (torch.optim.AdamW(model.parameters(),lr=1e-5), None)
)

In [9]:
trainer.train()

***** Running training *****
  Num examples = 904
  Num Epochs = 40
  Instantaneous batch size per device = 20
  Total train batch size (w. parallel, distributed & accumulation) = 100
  Gradient Accumulation steps = 5
  Total optimization steps = 360
  Number of trainable parameters = 355871744


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=360, training_loss=3.2786461724175346, metrics={'train_runtime': 824.2354, 'train_samples_per_second': 43.871, 'train_steps_per_second': 0.437, 'total_flos': 2098631394459648.0, 'train_loss': 3.2786461724175346, 'epoch': 39.98})

In [10]:
model.save_pretrained('model_mondelshtam')

Configuration saved in model_mondelshtam/config.json
Configuration saved in model_mondelshtam/generation_config.json
Model weights saved in model_mondelshtam/pytorch_model.bin


In [11]:
text = "Пришел на праздник "
input_ids = tokenizer.encode(text, return_tensors="pt").to('cuda:0')
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        num_beams=3,
                        temperature=1.5,
                        top_p=2.2,
                        max_length=100)

generated_text = list(map(tokenizer.decode, out))[0]

print(generated_text)

Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Пришел на праздник 
Монах печальный, усталый,
Вынул из-за пазухи 
Трехрублевую монету.

Увидел девушку простую,
Серую от пыли, бедную,
Взял монету и вышел
В чистое летнее утро.

Прямо к обрыву спустился,
Где голубая вода
И кудрявая осока,
Где поют соловьи.

Присел отдохнуть на камень,
Отдох
